Ценообразование кредитных спредов с помощью
Python

План
состоит в том, чтобы продать опционы на покупку денег, а затем купить их по более
высокой ставке, за вычетом разницы в виде прибыли. Чтобы застраховать эту позицию, я
куплю базовую акцию.
Теперь я хочу показать, как я искал возможности в рыночных данных. Существует
множество конкурирующих факторов, поэтому мне придется сопоставить их друг с другом.
Я хочу:
Максимальный авансовый взнос
Коэффициенты покупки / продажи, близкие к нулю Δ и Γ с большим положительным Θ
Минимальные требования к наличности при покупке акций и марже опционов
Волатильность акций на средне-низком уровне
Достаточная ликвидность акций и опционов
Чтобы упростить поиск и минимизировать риск, я собираюсь сосредоточиться на спредах
опционов между двумя немедленными ударами. Это также снизит наши маржинальные
издержки, хотя разница в премиях также будет меньше. Чтобы ограничить временной
горизонт, я собираюсь сосредоточиться на покупке спреда за неделю до истечения срока
действия опционов, надеясь, что срок их действия обесценится. Затем планируется поиск
следующего набора опционов для включения в эту сделку на следующей неделе.

In [10]:
# Standard library imports
import datetime
import re

# Требования к сторонним библиотекам
import pandas as pd
import QuantLib as ql
from yahoo_fin.options import get_calls
from yahoo_fin.stock_info import get_quote_table

In [14]:
# Жестко запрограммированный тикер и срок действия для целей примера
ticker = "AAPL"
expiration = datetime.date(2023, 2, 24)

In [12]:
# Получить текущую цену акций и ставку дивидендов
info = get_quote_table(ticker)
current_price = info["Quote Price"]
yield_re = re.compile(r"\((?P<value>(\d+\.\d+))%\)")
try:
    dividend_yield = float(
    yield_re.search(info["Forward Dividend & Yield"])["value"]
    )
except (KeyError, ValueError, TypeError):
    dividend_yield = 0.0

In [15]:
# Выборка цепочек цен на опционы call
calls = get_calls(ticker, expiration.strftime("%B %d, %Y"))

In [17]:
# Настройка инструментов для ценообразования по Блэку-Шоулзу
today = ql.Date.todaysDate()
underlying = ql.SimpleQuote(current_price)
exercise = ql.AmericanExercise(
today,
ql.Date(expiration.day, expiration.month, expiration.year)
)
dividendYield = ql.FlatForward(
today, dividend_yield, ql.Actual360()
)

riskFreeRate = ql.FlatForward(today, 0.0008913, ql.Actual360())

def create_option(row):
    volatility = ql.BlackConstantVol(
    today,
    ql.UnitedStates(),
    row["volatility"],
    ql.Business252()
    )
    option = ql.VanillaOption(
    ql.PlainVanillaPayoff(ql.Option.Call, row["Strike"]),
    exercise
    )
    process = ql.BlackScholesMertonProcess(
    ql.QuoteHandle(underlying),
    ql.YieldTermStructureHandle(dividendYield),
    ql.YieldTermStructureHandle(riskFreeRate),
    ql.BlackVolTermStructureHandle(volatility),
    )
    # Не используйте подразумеваемый объем в кавычках
    # Вычислите его изпоследняя цена
    imp_vol = option.impliedVolatility(row["Last Price"], process)
    implied_volatility = ql.BlackConstantVol(
    today,
    ql.UnitedStates(),
    imp_vol,
    ql.Business252()
    )

    process = ql.BlackScholesMertonProcess(
    ql.QuoteHandle(underlying),
    ql.YieldTermStructureHandle(dividendYield),
    ql.YieldTermStructureHandle(riskFreeRate),
    ql.BlackVolTermStructureHandle(implied_volatility),
    )
    option.setPricingEngine(
    ql.FdBlackScholesVanillaEngine(process, 1000, 1000)
    )
    return {
        "Name": row["Contract Name"],
        "Strike": row["Strike"],
        "Last": row["Last Price"],
        "Bid": row["Bid"],
        "Ask": row["Ask"],
        "NPV": option.NPV(),
        "Delta": option.delta(),
        "Gamma": option.gamma(),
        "Theta": option.theta() / 365,
        "Volatility": imp_vol * 100,
        }

    # Filter down to only OTM strikes
    calls = calls[calls["Strike"] >= current_price * 1.025]
    calls = calls[calls["Strike"] <= current_price * 1.10]
    # Parse out implied volatility
    calls = calls.assign(
    volatility=lambda x: x["Implied Volatility"].str.rstrip("%").astype("float") /
    100,
    )
    # Price options and calculate greeks
    options = calls.apply(create_option, axis=1, result_type="expand")

Минимизация риска и максимизация прибыли

Имея рыночные данные, а теперь и греческие значения для каждого опциона, я начинаю
оценивать каждую пару опционов, пытаясь найти ту, которая имеет наилучшее значение.
Опять же, я ограничиваю это только страйками, которые находятся рядом друг с другом в
цепочке, но это может быть расширено для поиска более широких спредов.